<a href="https://colab.research.google.com/github/vitoraugusto1993/Coursera-courses/blob/main/LLM/Hugging%20Face/Fine%20Tuning/Twitter%20Sentiment%20Analysis/LoRA_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pandas numpy transformers peft datasets evaluate torch accelerate>=0.26.0 psutil mlflow==3.1.1 mlflow-skinny==3.1.1 nltk pandas==2.2.2 numpy==2.0 uuid BitsAndBytes

In [ ]:
import re
import torch
from torch.utils.data import DataLoader
import mlflow
import evaluate
import random
import numpy as np
import pandas as pd
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding, AutoModelForSequenceClassification, AutoTokenizer
from transformers.integrations import MLflowCallback
import datasets
from datasets import Dataset
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random # Random module for generating random numbers and selections
import nltk
nltk.download('wordnet') # NLTK's WordNet corpus for finding synonyms
from nltk.corpus import wordnet

In [ ]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [ ]:
!pip install pyngrok --quiet

from pyngrok import ngrok
from getpass import getpass

# Terminate open tunnels if exist
# ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
# NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token('33ErQ33bYRKROHRmL4nPUnUVlXN_6Pr3nWEtBMuxD8wkbsEUw')

# O pen an HTTPs tunnel on port 5000 for http://localhost:5000
# 33ErQ33bYRKROHRmL4nPUnUVlXN_6Pr3nWEtBMuxD8wkbsEUw
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

## Step 1: Import data set

Import dataset from Hugging Face Hub. This Dataset contains thousands of tweets classified as Negative (0) and Positive (1).

In [ ]:
from datasets import load_dataset

# data = load_dataset("gxb912/large-twitter-tweets-sentiment")

data = datasets.load_dataset('sayanroy058/Twitter_Tweets_Mental_Pressure_Sentiment_Dataset')
data = data.remove_columns(column_names=['textID','selected_text'])
dataset = data['train'].to_pandas()
dataset.head()

## Step 2: Clean the text

This step is cleaning the raw text data to remove unnecessary characters, such as URLs, special symbols, or HTML tags, and to normalize the text by converting it to lowercase.



In [ ]:
# Function to clean the text
def clean_text(text):
    text = str(text).lower() # Convert all text to lowercase for uniformity
    text = re.sub(r'http\S+', '', text) # Remove URLs from the text
    text = re.sub(r'<.*?>', '', text) # Remove any HTML tags from the text
    text = re.sub(r'@\S+ ', '', text) # Remove any user account mention (e.g: @user)
    # text = re.sub(r'[^\w\s]', '', text) # Remove punctuation, keep only words and spaces
    return text # Return the cleaned text

# Assume `data` is a pandas DataFrame with a column named 'text'
# Apply the cleaning function to each row of the 'text' column
dataset['cleaned_text'] = dataset['text'].apply(clean_text)

# Map sentiment into numerical categories
categories = {'neutral': 0, 'negative': 1, 'positive': 2}
dataset['Labels'] = dataset['sentiment'].map(categories)

# Remove un-necessary columns
dataset = dataset.drop(['text','sentiment'], axis=1)

# Print the first 5 rows of the cleaned text to verify the cleaning process
print(dataset.head())

## Step 3: Handle missing data

Check for empty entries in the column 'Text'. If there are missing data, we should delete the entire rows.

In [ ]:
# Print the count of missing values for each column
print(dataset.isnull().sum())
print('--------------------')

# Remove rows with missing data in the 'text' column
dataset = dataset.dropna()

# Confirmation of no missing data in the dataset
print(dataset.isnull().sum())
print('--------------------')

# Print the count of entries in the dataset
print(dataset.info())

## Step 4: Data augmentation

In certain cases, especially when data is limited, data augmentation techniques can be applied to generate new training examples by modifying the original dataset.

- Paraphrasing: rewriting sentences in different ways while preserving the meaning

- Backtranslation: translating text into another language and back again to create variation

- Synonym replacement: replacing certain words in the text with their synonyms



In [ ]:
dataset.groupby('Labels')['cleaned_text'].count()

In [ ]:
# Define a function to find and replace a word with a synonym
def synonym_replacement(word):
# Get all synsets (sets of synonyms) for the given word from WordNet
    synonyms = wordnet.synsets(word)

    # If the word has synonyms, randomly choose one synonym, otherwise return the original word
    # Select a random synonym and get the first lemma (word form) of that synonym
    # If no synonyms are found, return the original word
    if synonyms:
        return random.choice(synonyms).lemmas()[0].name()
    return word

# Define a function to augment text by replacing words with synonyms randomly
def augment_text(example, alpha):
    # Split the input text into individual words
    words = example[0].split() # Split the input text into individual words

    if random.random() > (1 - alpha):
        # Replace each word with a synonym with a probability of 20% (random.random() > 0.8)
        augmented_words = [synonym_replacement(word) if random.random() > 0.8 else word
                            # If random condition met, replace
                            for word in words] # Iterate over each word in the original text

        # Join the augmented words back into a single string and return it
        return pd.Series([' '.join(augmented_words), int(example[1])])
    else:
        return pd.Series([None, None])

In [ ]:
# Apply the text augmentation function to the 'cleaned_text' column in a DataFrame
# Create a new column 'augmented_text' containing the augmented version of 'cleaned_text'
# Apply about 40% augmentation on samples labeled as Negative (1) and include augmented examples in the original dataset
dataset_aug = dataset[dataset['Labels'] == 1].apply(augment_text, alpha=0.4, axis=1)
dataset_aug.columns=['cleaned_text','Labels']
dataset_aug = dataset_aug.dropna()
dataset = pd.concat([dataset, dataset_aug], axis=0, ignore_index=True)
# Apply about 30% augmentation on samples labeled as Positive (2) and include augmented examples in the original dataset
dataset_aug = dataset[dataset['Labels'] == 2].apply(augment_text, alpha=0.3, axis=1)
dataset_aug.columns=['cleaned_text','Labels']
dataset_aug = dataset_aug.dropna()
dataset = pd.concat([dataset, dataset_aug], axis=0, ignore_index=True)

dataset['Labels'] = dataset['Labels'].astype(int)
dataset.head()

In [ ]:
dataset.groupby('Labels')['cleaned_text'].count()

In [ ]:
# Visually check if data is balanced
dataset["Labels"].hist()

## Step 5: Tokenization

After cleaning the text, we tokenize it. Tokenization splits the text into individual words or subwords that can be used by the model. We will use the BERT tokenizer to ensure compatibility with the Brie-trained model you are fine-tuning.

In [ ]:
random_indices = random.sample(range(len(dataset)), 70000)
mini_dataset = Dataset.from_pandas(dataset).select(random_indices)
mini_dataset = Dataset.to_pandas(mini_dataset)
mini_dataset.groupby('Labels')['cleaned_text'].count()

In [ ]:
# Load the BERT tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# dataset_ds = Dataset.from_pandas(mini_dataset)
dataset_ds = Dataset.from_pandas(dataset)

def tokenize_function(examples):
    return tokenizer(examples['cleaned_text'], truncation=True)

tokenized_dataset = dataset_ds.map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(['cleaned_text'])
tokenized_dataset = tokenized_dataset.rename_column('Labels', 'labels')
tokenized_dataset.set_format("torch")
tokenized_dataset.column_names

## Step 6: Structure the data for fine-tuning

You can fine-tune your model once the dataset is cleaned and tokenized. The next step is structuring the data for fine-tuning.

First we will split our dataset into three separate datasets: training, validation and test. For this case, we will use 70% of the whole dataset to train de model, 15% to validade and 15% to test. Then, we need to convert our pandas dataframes back to Datasets objects. With our dataset splited and converted, we have to remove an extra column that these steps added ('__index_level_0__')



In [ ]:
tokenized_df = tokenized_dataset.to_pandas()

train_df, val_test_df = train_test_split(
    tokenized_df, test_size=0.3, random_state=42
)

val_df, test_df = train_test_split(
    val_test_df, test_size=0.5, random_state=42
)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.remove_columns(['__index_level_0__'])
val_dataset = val_dataset.remove_columns(['__index_level_0__'])
test_dataset = test_dataset.remove_columns(['__index_level_0__'])

When we load a dataset using the load_dataset from library datasets, they are presented as a DatasetDict object. Sometimes the dataset is already splited into train, validation and test. So, in the next step, we will create a DatasetDict object just like the ones we get when me use load_data function.

In [ ]:
from datasets import Dataset, DatasetDict

tweet_dataset = DatasetDict({"train": train_dataset, "val": val_dataset, "test":test_dataset})
tweet_dataset

## Step 7: Create the model

Now that we're completely finished with data preprocessing, let's turn to the model. We instantiate it by referencing the pretrained model we want to fine tune. In this case e are using a base DistilBERT model. In this example, we will implement a LoRA (Low-Rank Adaptation) fine-tuning method. LoRA is a parameter-efficient fine-tuning (PEFT) technique that adapts large, pre-trained machine learning models for specific tasks by adding and training only a small number of new parameters. Instead of retraining the entire model, LoRA injects lightweight, low-rank matrices into the original model's layers, freezing the original weights and creating efficient, task-specific adapters. This significantly reduces computational costs and memory requirements, allowing for faster customization of massive models like Large Language Models (LLMs) without compromising performance, and enabling easy switching between different tasks by swapping the adapter weights.

How LoRA Works
* Freeze the base model: The original, large pre-trained model's weights are kept fixed and are not modified during the adaptation process.
* Inject LoRA adapters: Instead of updating the original weight matrices (ΔW) directly, LoRA decomposes the necessary weight update into two smaller, low-rank matrices (A and B).
* Train only the adapters: The adaptation process then focuses on training only these smaller A and B matrices.
* Merge for inference (optional): For deployment, the trained adapter matrices can be merged back into the base model's weights, creating a single, task-specific model without any added latency.

We can simply implement LoRA with the Hugging Face's library PEFT by instantiating a LoraConfig object with the desired parameters. Then, we apply this configuration to the model through the method get_peft_model.

In [ ]:
from peft import LoraConfig, get_peft_model

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3,)

# Print all model modules to inspect and select the target modules to apply LoRA
# for idx, m in enumerate(model.named_modules()):
#   print(idx, '->', m)

rank = 8

lora_config = LoraConfig(
        r=rank,
        lora_alpha=rank*2,
        target_modules=['v_lin','q_lin','k_lin'],
        lora_dropout=0.1,
        bias='none',
        use_rslora=True,
        task_type="SEQ_CLS" # Specify the task type
    )

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters() # To see the number of trainable parameters

An optional, but important thing to do before training is to set a GPU to do the work. If we have access to one, training process can take just a couple minutes instead of several ours compared to CPU. To do this, we define a device we will put our model and our batches on:

In [ ]:
# Using a GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
lora_model.to(device)
device

## Step 8: Training loop configuration

The Hugging Face's transformers Trainer Class provides a high level API for training and evaluating PyTorch models, particularly those based on the transformers library. It simplifies the process of setting up and running training loops, handling complexities like distributed training, mixed-precision training, logging and model checkpointing, allowing users to focus on model architecture, datasets and training configuration through the TrainingArguments class.

To use the Trainer, you tipically:
*   Define your model, dataset, and preprocessor (e.g., tokenizer);
*   Create an instance of TrainingArguments to configure your training parameters;
*   Instantiate the Trainer with your model, TrainingArguments, datasets, and preprocessor;
*   Call the train() method to start the training process.

The Trainer streamlines the process of fine-tuning pre-trained transformers models or training new models from scratch, making it a powerful toll for researchers and developers working with NLP and other machine learning tasks.

#### Evaluation Metrics

By default, the Trainer only tracks the loss function during training and evaluation. To track how the model is performing with the validation data we need to specify an evaluation metric, such as accuracy, recall, precision and F1-score.

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # For multi-class classification, convert logits to predicted class IDs
    predictions = np.argmax(predictions, axis=1)

    # Calculate accuracy, precision, recall, and f1-score.
    accuracy = evaluate.load("accuracy")
    precision = evaluate.load("precision")
    recall = evaluate.load("recall")
    f1 = evaluate.load("f1")

    # 'average' can be 'binary', 'micro', 'macro', 'weighted', or None.
    # 'weighted' is often suitable for imbalanced datasets in multi-class.
    metrics = {}
    metrics.update(accuracy.compute(predictions=predictions, references=labels))
    metrics.update(precision.compute(predictions=predictions, references=labels, average='weighted'))
    metrics.update(recall.compute(predictions=predictions, references=labels, average='weighted'))
    metrics.update(f1.compute(predictions=predictions, references=labels, average='weighted'))

    return metrics

#### Training

With the evaluation metrics defined, we can now configure and run the model training.

In [ ]:
from uuid import uuid4
import mlflow

lr = 3e-4
num_epochs = 5
batch_size = 8

mlflow.set_experiment(f"LoRA_NNP_Twitter_{model_name}")
mlflow.pytorch.autolog()
run_id = f"model-lora-rank-{rank}-lr{str(lr)}-batch-size-{batch_size}-{num_epochs}-epochs-{str(uuid4())[:8]}"

mlflow.end_run()
with mlflow.start_run(log_system_metrics=True, run_name=run_id) as run:
  training_args = TrainingArguments(        # Step 1: Set training arguments for fine-tuning the model
    output_dir='./results',                 # Directory where results will be stored
    num_train_epochs=num_epochs,            # Number of epochs (full passes through the dataset)
    per_device_train_batch_size=batch_size, # Batch size per GPU/CPU during training
    per_device_eval_batch_size=batch_size,  # Batch size per GPU/CPU during evaluation
    eval_strategy="epoch",                  # Evaluate the model at the end of each epoch
    optim="adamw_torch",                    # Define the optimizer
    lr_scheduler_type="linear",             # Define the behavior os the decay of learning rate
    learning_rate=lr,                       # Learning rate
    weight_decay=0.01,                      # Learning rate decay of the training steps
    metric_for_best_model="accuracy",       # Set the metric que trainer will use to evaluate the model
    greater_is_better=True,                 # How the trainer should evaluate a model as better or worse than others
    load_best_model_at_end=True,            # Load the best model at the end of training
    save_strategy='epoch',                  # Save model at the end of an epoch with best evaluation metric
    report_to="mlflow"                      # Enable MLflow logging
  )

  trainer = Trainer(                        # Step 2: Fine-tune only the final classification head (since earlier layers were frozen)
    model=lora_model,                            # Pre-trained BERT model with frozen layers
    args=training_args,                     # Training arguments
    train_dataset=tweet_dataset['train'],   # Training data for fine-tuning
    eval_dataset=tweet_dataset['val'],      # Validation data to evaluate performance during training
    data_collator=data_collator,            # Data collator to prepare batches of data
    processing_class=tokenizer,             # Tokenizer to preprocess the data
    compute_metrics=compute_metrics,        # Function to compute evaluation metrics
  )

  # Step 3: Train the model using PEFT (this performs PEFT because layers were frozen in Step 1)
  trainer.train()

In [ ]:
mlflow.end_run()

## Step 9: Evaluation

In [ ]:
trainer.evaluate(eval_dataset=tweet_dataset['test'])